In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 15.4 MB/s eta 0:00:00


In [3]:
!pip install pandas_path

In [4]:
!pip install pytorch-lightning==0.7.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [5]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227141 sha256=fec8c15a96d1fa42ac94eaaf88d704b32fc8c64c9829db642e348ca7b534c785
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [6]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 5.3 MB/s eta 0:00:00


In [7]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 39.7 MB/s eta 0:00:00


In [8]:

%matplotlib inline

import os
import json
import logging
from pathlib import Path
import random
import tarfile
import tempfile
import warnings

import torch
import torchvision
import fasttext
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas_path import path
from tqdm import tqdm
import pytorch_lightning as pl

In [9]:
SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [10]:
import nltk
nltk.download('stopwords')
import contractions


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
import os
import telebot
import easyocr
import numpy as np
import cv2

reader = easyocr.Reader(['en'])
# Replace with your bot token and desired image save path
TOKEN = '6633194857:AAGFUjlf8NrFKjXVG64R3inWyR9sgYvWKKk'
IMAGE_SAVE_PATH = '/content/drive/MyDrive/major project/data/img'
JSON_SAVE_PATH = '/content/drive/MyDrive/major project/data'

bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.reply_to(message, "Hello! I'm a bot. How can I help you?")

@bot.message_handler(commands=['help'])
def handle_help(message):
    bot.reply_to(message, "You can send me a message and I'll try to respond. I'm still learning, so please be patient with me!")

@bot.message_handler(commands=['show_text'])
def handle_help(message):
    photo = message.photo[-1]
    file_info = bot.get_file(photo.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    filename = os.path.join(IMAGE_SAVE_PATH, f"{photo.file_id}.png")
    with open(filename, 'wb') as f:
        f.write(downloaded_file)
    result = reader.readtext(downloaded_file)
    # Extract and format the text
    text = '\n'.join([item[1] for item in result])
    bot.reply_to(message, text)

@bot.message_handler(content_types=['photo'])
def photo(message):
    photo = message.photo[-1]
    file_info = bot.get_file(photo.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    # Perform OCR using easyocr
    filename = os.path.join(IMAGE_SAVE_PATH, f"{photo.file_id}.png")
    with open(filename, 'wb') as f:
        f.write(downloaded_file)
    result = reader.readtext(downloaded_file)
    # Extract and format the text
    # text = '\n'.join([item[1] for item in result])
    text = ' '.join([item[1] for item in result])
    text = text.strip()
    print(text)
    data = {
        "id":16395,"img":f"img/{photo.file_id}.png","text":text
    }
    jsonl_filename = os.path.join(JSON_SAVE_PATH, f"{photo.file_id}.jsonl")
    with open(jsonl_filename, 'w', newline='\n') as jsonl_file:
        json.dump(data, jsonl_file)
    data_dir = '/content/drive/MyDrive/major project/data/'
    img_path = data_dir + "data_dir"
    test_path = jsonl_filename
    train_path = data_dir + "train.jsonl"
    dev_path = data_dir + "dev.jsonl"
    print(train_path)
    print(test_path)
    print(img_path)
    test_samples_frame = pd.read_json(test_path, lines=True)
    print(test_samples_frame.shape)
    test_samples_frame.head()

    from PIL import Image
    image = Image.open(
            filename
        ).convert("RGB")
    print(image.size)
    # define a callable image_transform with Compose
    image_transform = torchvision.transforms.Compose(
        [
            torchvision.transforms.Resize(size=(224, 224)),
            torchvision.transforms.ToTensor()
        ]
    )

    # convert the images and prepare for visualization.
    tensor_img = torch.stack(
        [image_transform(image)]
    )
    grid = torchvision.utils.make_grid(tensor_img)

    # plot
    plt.rcParams["figure.figsize"] = (20, 5)
    plt.axis('off')
    _ = plt.imshow(grid.permute(1, 2, 0))
    from unicodedata import normalize, category

    def clean_text_1(_text):
        _text = contractions.fix(_text).lower().replace('(', '( ').replace(')', ' ) ').replace('\n', '')

        selects_characters = ['Ll', 'Zs', 'Lu'] #, 'Nd'
        return ''.join(
            [_ for _ in normalize('NFD', _text) if category(_) in selects_characters]
        ).split()

    def quit_stopwords(_text):
        return ' '.join([word for word in _text if word not in stop_words and len(word) > 2])
    stop_words = nltk.corpus.stopwords.words('english')

    stop_words = list(set(stop_words + contractions.fix(' '.join(stop_words)).lower().split()))
    print(stop_words)
    value = np.random.choice([0,1])


    class HatefulMemesDataset(torch.utils.data.Dataset):
        def __init__(
            self,
            data_path,
            img_dir,
            image_transform,
            text_transform,
            balance=False,
            dev_limit=None,
            random_state=SEED,
        ):

            self.samples_frame = pd.read_json(
                data_path, lines=True
            )
            self.dev_limit = dev_limit
            if balance:
                neg = self.samples_frame[
                    self.samples_frame.label.eq(0)
                ]
                pos = self.samples_frame[
                    self.samples_frame.label.eq(1)
                ]
                self.samples_frame = pd.concat(
                    [
                        neg.sample(
                            pos.shape[0],
                            random_state=random_state
                        ),
                        pos
                    ]
                )
            if self.dev_limit:
                if self.samples_frame.shape[0] > self.dev_limit:
                    self.samples_frame = self.samples_frame.sample(
                        dev_limit, random_state=random_state
                    )
            self.samples_frame = self.samples_frame.reset_index(
                drop=True
            )
            self.samples_frame.img = self.samples_frame.apply(
                lambda row: (img_dir + row.img), axis=1
            )
            print("Image paths:")
            print(self.samples_frame.img)
            # Check if image files exist
            invalid_paths = self.samples_frame[~self.samples_frame.img.apply(lambda x: os.path.exists(x))]
            if not invalid_paths.empty:
                # Remove extra backslashes from paths
                self.samples_frame.img = self.samples_frame.img.str.replace(r'\\', '/')
                # Recheck for invalid paths
                invalid_paths = self.samples_frame[~self.samples_frame.img.apply(lambda x: os.path.exists(x))]
                if not invalid_paths.empty:
                    print("Invalid image paths:")
                    print(invalid_paths.img)
                    raise FileNotFoundError("One or more image files do not exist.")

            self.image_transform = image_transform
            self.text_transform = text_transform

        def __len__(self):
            """This method is called when you do len(instance)
            for an instance of this class.
            """
            return len(self.samples_frame)

        def __getitem__(self, idx):
            """This method is called when you do instance[key]
            for an instance of this class.
            """
            if torch.is_tensor(idx):
                idx = idx.tolist()

            img_id = self.samples_frame.loc[idx, "id"]

            image = Image.open(
                self.samples_frame.loc[idx, "img"]
            ).convert("RGB")
            image = self.image_transform(image)

            text = torch.Tensor(
                self.text_transform.get_sentence_vector(
                    self.samples_frame.loc[idx, "text"]
                )
            ).squeeze()

            if "label" in self.samples_frame.columns:
                label = torch.Tensor(
                    [self.samples_frame.loc[idx, "label"]]
                ).long().squeeze()
                sample = {
                    "id": img_id,
                    "image": image,
                    "text": text,
                    "label": label
                }
            else:
                sample = {
                    "id": img_id,
                    "image": image,
                    "text": text
                }

            return sample

    class LanguageAndVisionConcat(torch.nn.Module):
        def __init__(
            self,
            num_classes,
            loss_fn,
            language_module,
            vision_module,
            language_feature_dim,
            vision_feature_dim,
            fusion_output_size,
            dropout_p,

        ):
            super(LanguageAndVisionConcat, self).__init__()
            self.language_module = language_module
            self.vision_module = vision_module

            print(language_feature_dim)
            print(vision_feature_dim)
            assert language_feature_dim == vision_feature_dim

            self.fusion = torch.nn.Linear(
                in_features=vision_feature_dim,
                out_features=fusion_output_size
            )
            self.fc = torch.nn.Linear(
                in_features=fusion_output_size,
                out_features=num_classes
            )
            self.loss_fn = loss_fn
            self.dropout = torch.nn.Dropout(dropout_p)

        def forward(self, text, image, label=None):
            text_features = torch.nn.functional.relu(
                self.language_module(text)
            )
            image_features = torch.nn.functional.relu(
                self.vision_module(image)
            )


            combined = torch.mul(text_features, image_features)

            fused = self.dropout(
                torch.nn.functional.relu(
                self.fusion(combined)
                )
            )
            logits = self.fc(fused)
            pred = torch.nn.functional.softmax(logits)
            loss = (
                self.loss_fn(pred, label)
                if label is not None else label
            )
            return (pred, loss)
    x = torch.randn(2, 3)
    # for the purposes of this post, we'll filter
    # much of the lovely logging info from our LightningModule
    #warnings.filterwarnings("ignore")
    logging.getLogger().setLevel(logging.INFO)
    class HatefulMemesModel(pl.LightningModule):
        def __init__(self, hparams):
            for data_key in ["train_path", "dev_path", "img_dir",]:
                # ok, there's one for-loop but it doesn't count
                if data_key not in hparams.keys():
                    raise KeyError(
                        f"{data_key} is a required hparam in this model"
                    )

            super(HatefulMemesModel, self).__init__()
            self.hparams = hparams

            # assign some hparams that get used in multiple places
            self.embedding_dim = self.hparams.get(
                "embedding_dim", 500
            )
            self.language_feature_dim = self.hparams.get(
                "language_feature_dim", 500
            )
            self.vision_feature_dim = self.hparams.get(
                "vision_feature_dim", 500
            )
            self.output_path = Path(
                self.hparams.get("output_path", "model-outputs")
            )
            self.output_path.mkdir(exist_ok=True)

            # instantiate transforms, datasets
            self.text_transform = self._build_text_transform()
            self.image_transform = self._build_image_transform()
            self.train_dataset = self._build_dataset("train_path")
            self.dev_dataset = self._build_dataset("dev_path")

            # set up model and training
            self.model = self._build_model()
            self.trainer_params = self._get_trainer_params()


        def forward(self, text, image, label=None):
            return self.model(text, image, label)

        def training_step(self, batch, batch_nb):
            preds, loss = self.forward(
                text=batch["text"],
                image=batch["image"],
                label=batch["label"]
            )

            return {"loss": loss}

        def validation_step(self, batch, batch_nb):
            preds, loss = self.eval().forward(
                text=batch["text"],
                image=batch["image"],
                label=batch["label"]
            )

            return {"batch_val_loss": loss}

        def validation_epoch_end(self, outputs):
            avg_loss = torch.stack(
                tuple(
                    output["batch_val_loss"]
                    for output in outputs
                )
            ).median()

            return {
                "val_loss": avg_loss,
                "progress_bar":{"avg_val_loss": avg_loss}
            }

        def configure_optimizers(self):
            optimizers = [
                torch.optim.AdamW(
                    self.model.parameters(),
                    lr=self.hparams.get("lr", 0.001)
                )
            ]
            schedulers = [
                torch.optim.lr_scheduler.ReduceLROnPlateau(
                    optimizers[0],
                    factor=0.1,
                    patience=2
                )
            ]
            return optimizers, schedulers

        @pl.data_loader
        def train_dataloader(self):
            return torch.utils.data.DataLoader(
                self.train_dataset,
                shuffle=True,
                batch_size=self.hparams.get("batch_size", 4),
                num_workers=self.hparams.get("num_workers", 16)
            )

        @pl.data_loader
        def val_dataloader(self):
            return torch.utils.data.DataLoader(
                self.dev_dataset,
                shuffle=False,
                batch_size=self.hparams.get("batch_size", 4),
                num_workers=self.hparams.get("num_workers", 16)
            )

        ## Convenience Methods ##

        def fit(self):
            self._set_seed(self.hparams.get("random_state", SEED))
            self.trainer = pl.Trainer(**self.trainer_params)
            self.trainer.fit(self)

        def _set_seed(self, seed=SEED):
            random.seed(seed)
            np.random.seed(seed)


            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(seed)
        def _build_text_transform(self):
            with tempfile.NamedTemporaryFile() as ft_training_data:
                ft_path = Path(ft_training_data.name)
                with ft_path.open("w") as ft:
                    training_data = [
                        quit_stopwords(clean_text_1(json.loads(line)["text"])) + "/n"
                        for line in open(
                            self.hparams.get("train_path")
                        ).read().splitlines()
                    ]
                    for line in training_data:
                        ft.write(line + "\n")
                    language_transform = fasttext.train_unsupervised(
                        str(ft_path),
                        model=self.hparams.get("fasttext_model", "skipgram"),
                        dim=self.embedding_dim,
                        epoch=20,
                        lr=0.001
                    )
            return language_transform

        def _build_image_transform(self):
            image_dim = self.hparams.get("image_dim", 224)
            image_transform = torchvision.transforms.Compose(
                [
                    torchvision.transforms.Resize(
                        size=(image_dim, image_dim)
                    ),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(
                        mean=(0.485, 0.456, 0.406),
                        std=(0.229, 0.224, 0.225)
                    ),
                ]
            )
            return image_transform

        def _build_dataset(self, dataset_key):
            return HatefulMemesDataset(
                data_path=self.hparams.get(dataset_key, dataset_key),
                img_dir=self.hparams.get("img_dir"),
                image_transform=self.image_transform,
                text_transform=self.text_transform,
                # limit training samples only
                dev_limit=(
                    self.hparams.get("dev_limit", None)
                    if "train" in str(dataset_key) else None
                ),
                balance=True if "train" in str(dataset_key) else False,
            )

        def _build_model(self):

            language_module = torch.nn.Linear(
                    in_features=self.embedding_dim,
                    out_features=self.language_feature_dim
            )


            vision_module = torchvision.models.resnet152(
                pretrained=True
            )
            vision_module.fc = torch.nn.Linear(
                    in_features=2048,
                    out_features=self.vision_feature_dim
            )

            return LanguageAndVisionConcat(
                num_classes=self.hparams.get("num_classes", 2),
                loss_fn=torch.nn.CrossEntropyLoss(),
                language_module=language_module,
                vision_module=vision_module,
                language_feature_dim=self.language_feature_dim,
                vision_feature_dim=self.vision_feature_dim,
                fusion_output_size=self.hparams.get(
                    "fusion_output_size", 512
                ),
                dropout_p=self.hparams.get("dropout_p", 0.1),
            )

        def _get_trainer_params(self):
            checkpoint_callback = pl.callbacks.ModelCheckpoint(
                filepath=self.output_path,
                monitor=self.hparams.get(
                    "checkpoint_monitor", "avg_val_loss"
                ),
                mode=self.hparams.get(
                    "checkpoint_monitor_mode", "min"
                ),
                verbose=self.hparams.get("verbose", True)
            )

            early_stop_callback = pl.callbacks.EarlyStopping(
                monitor=self.hparams.get(
                    "early_stop_monitor", "avg_val_loss"
                ),
                min_delta=self.hparams.get(
                    "early_stop_min_delta", 0.0001
                ),
                patience=self.hparams.get(
                    "early_stop_patience", 5
                ),
                verbose=self.hparams.get("verbose", True),
            )

            trainer_params = {
                "checkpoint_callback": checkpoint_callback,
                "early_stop_callback": early_stop_callback,
                "default_save_path": self.output_path,
                "accumulate_grad_batches": self.hparams.get(
                    "accumulate_grad_batches", 1
                ),
                "gpus": self.hparams.get("n_gpu", 1),
                "max_epochs": self.hparams.get("max_epochs", 100),
                "gradient_clip_val": self.hparams.get(
                    "gradient_clip_value", 1
                ),
            }
            return trainer_params

        @torch.no_grad()
        def make_submission_frame(self, test_path):
            test_dataset = self._build_dataset(test_path)
            print(test_dataset)
            print(type(test_dataset))
            print(dir(test_dataset))
            print("/"*100)
            print(dir(test_dataset.samples_frame))
            print("/"*100)
            print(test_dataset.samples_frame.id)
            print(type(test_dataset.samples_frame.id))

            submission_frame = pd.DataFrame(
                index=test_dataset.samples_frame.id,
                columns=["proba", "label_predict"]
            )
            # print(submission_frame.head(10))

            test_dataloader = torch.utils.data.DataLoader(
                test_dataset,
                shuffle=False,
                batch_size=self.hparams.get("batch_size", 4),
                num_workers=self.hparams.get("num_workers", 16))

            for batch in tqdm(test_dataloader, total=len(test_dataloader)):

                preds, _ = self.model.eval()(
                    batch["text"], batch["image"]
                )
                submission_frame.loc[batch["id"], "proba"] = preds[:, 1]
                submission_frame.loc[batch["id"], "label_predict"] = preds.argmax(dim=1)

            submission_frame.proba = submission_frame.proba.astype(float)
            submission_frame.label_predict = submission_frame.label_predict.astype(int)
            return submission_frame
    checkpoints = list(Path("/content/drive/MyDrive/").glob("*.ckpt"))
    assert len(checkpoints) == 1
    checkpoints
    hateful_memes_model = HatefulMemesModel.load_from_checkpoint(
        checkpoints[0]
    )
    submission = hateful_memes_model.make_submission_frame(
        test_path
    )
    if ((submission['label_predict'] == 1) & (submission['proba'] > 0.5)).any() or ((submission['label_predict'] == 0) & (submission['proba'] < 0.4)).any():
        bot.reply_to(message, "Hateful")
    else:
        bot.reply_to(message, "Benign")
    print(submission)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
bot.polling()

X a smile iS worth a tousand words
/content/drive/MyDrive/major project/data/train.jsonl
/content/drive/MyDrive/major project/data/AgACAgUAAx0CYA6UGwAD_2YwcFNeTmlTxnSshEaY1l0XHEC9AAIhwDEbTN-ZVA307R7Pr2ScAQADAgADeQADNAQ.jsonl
/content/drive/MyDrive/major project/data/data_dir
(1, 3)
(1080, 714)
['own', 'be', 'because', 'same', 'of', 'might', 'for', 'wouldn', 'such', 'shall', "should've", 'the', 'nor', 'their', 're', 'before', 'what', 'll', "don't", "didn't", 'its', 'if', "hadn't", 'no', 'as', 'weren', 'some', 'he', 'by', 'all', 'above', 'it', 'at', 'yours', 'your', 'i', "isn't", 'both', 'him', "weren't", 'after', 'should', 'between', 'could', 'herself', 'this', 'now', 'was', 'haven', 'you', 'that', 'been', 'an', 'how', 'to', 'down', 'himself', 'why', 'were', 'who', "you're", 'out', 'yourself', 'so', 'these', 'has', 'me', 'in', 'my', 'have', 'once', "she's", 'aren', 'too', 'need', 'had', "wouldn't", 'other', 'shouldn', "hasn't", 'each', 'which', 'needn', 'whom', 'very', 'only', 'o', 'und